**TASK 4**

In [66]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

**Numerical Prediction**

In [67]:
#shift mood by 1 (since we're predicting for the day after)
data = pd.read_csv('linear_interpolation_data.csv')
data['mood'] = data['mood'].shift(1)
data['time'] = pd.to_datetime(data['time'])


In [68]:
data

,Unnamed: 0,id,time,activity,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,appCat.game,appCat.office,...,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,circumplex.arousal,circumplex.valence,mood,screen,sms
0,0,AS14.01,2014-02-17,0.000000,0.000,0.000,0.000,0.0,0.0,0.000,...,0.000,0.000,0.000,0.0,2.0,NaN,NaN,NaN,0.000000,0.0
1,1,AS14.01,2014-02-18,0.000000,0.000,0.000,0.000,0.0,0.0,0.000,...,0.000,0.000,0.000,0.0,1.0,NaN,NaN,NaN,0.000000,0.0
2,2,AS14.01,2014-02-19,0.000000,0.000,0.000,0.000,0.0,0.0,0.000,...,0.000,0.000,0.000,0.0,7.0,NaN,NaN,NaN,0.000000,2.0
3,3,AS14.01,2014-02-20,0.000000,0.000,0.000,0.000,0.0,0.0,0.000,...,0.000,0.000,0.000,0.0,2.0,NaN,NaN,NaN,0.000000,3.0
4,4,AS14.01,2014-02-21,0.000000,0.000,0.000,0.000,0.0,0.0,0.000,...,0.000,0.000,0.000,0.0,0.0,NaN,NaN,NaN,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,1968,AS14.33,2014-05-27,0.304902,726.391,2078.403,124.694,0.0,0.0,0.000,...,0.000,0.000,56.173,0.0,1.0,-0.600000,0.4,5.4,4089.879001,2.0
1969,1969,AS14.33,2014-05-28,2.479228,2551.046,1936.415,614.706,0.0,0.0,357.909,...,0.000,0.000,30.666,0.0,10.0,0.000000,1.2,6.2,14320.867998,1.0
1970,1970,AS14.33,2014-05-29,4.064507,400.034,435.105,136.738,0.0,0.0,0.000,...,0.939,0.000,3.199,0.0,5.0,-1.333333,1.0,8.2,3569.341000,1.0
1971,1971,AS14.33,2014-05-30,4.050925,3044.030,1670.842,260.220,0.0,0.0,0.000,...,1052.648,8.072,232.825,0.0,4.0,-0.800000,-0.4,7.0,9497.646999,0.0


In [51]:
#TASK 4 ALG 1 - GRADIENT BOOSTING REGRESSOR

#drop rows where mood = Na
data = data.dropna(subset=['mood'])
data['day_of_week'] = data['time'].dt.dayofweek
data.drop('time', axis=1, inplace=True)

#1hot encode IDs

encoder = OneHotEncoder()
encoded_ids = encoder.fit_transform(data[['id']]).toarray()
encoded_id_df = pd.DataFrame(encoded_ids, columns=["id_" + str(i) for i in range(encoded_ids.shape[1])])
data = pd.concat([data.reset_index(drop=True), encoded_id_df.reset_index(drop=True)], axis=1)
data.drop('id', axis=1, inplace=True)

#set up time series K fold split
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(data):

    #split
    train_data, test_data = data.iloc[train_index], data.iloc[test_index]

    #allocate features and target
    X_train = train_data.drop("mood", axis="columns")
    y_train = train_data["mood"]

    X_test = test_data.drop("mood", axis="columns")
    y_test = test_data["mood"]

    #model
    gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
    gbr.fit(X_train, y_train)

    #predict
    y_pred = gbr.predict(X_test)

    #evaluation 
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"Mean Absolute Error: {mae:.2f}")

    
    


/var/folders/gv/_x9lnydj1d71rl0vtl5mrtg00000gn/T/ipykernel_1550/2248557140.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['day_of_week'] = data['time'].dt.dayofweek
/var/folders/gv/_x9lnydj1d71rl0vtl5mrtg00000gn/T/ipykernel_1550/2248557140.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('time', axis=1, inplace=True)


Mean Squared Error: 0.82
Mean Absolute Error: 0.70
Mean Squared Error: 0.39
Mean Absolute Error: 0.47
Mean Squared Error: 0.44
Mean Absolute Error: 0.49
Mean Squared Error: 0.35
Mean Absolute Error: 0.46
Mean Squared Error: 0.35
Mean Absolute Error: 0.45


In [65]:
#TASK 4 ALG 1 - SUPPORT VECTOR REGRESSOR

#drop rows where mood = Na
data = data.dropna(subset=['mood'])
data['day_of_week'] = data['time'].dt.dayofweek
data.drop('time', axis=1, inplace=True)

#1hot encode IDs
encoder = OneHotEncoder()
encoded_ids = encoder.fit_transform(data[['id']]).toarray()
encoded_id_df = pd.DataFrame(encoded_ids, columns=["id_" + str(i) for i in range(encoded_ids.shape[1])])
data = pd.concat([data.reset_index(drop=True), encoded_id_df.reset_index(drop=True)], axis=1)
data.drop('id', axis=1, inplace=True)

#set up time series K fold split
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(data):

    #split
    train_data, test_data = data.iloc[train_index], data.iloc[test_index]

    #allocate features and target
    X_train = train_data.drop("mood", axis="columns")
    y_train = train_data["mood"]

    X_test = test_data.drop("mood", axis="columns")
    y_test = test_data["mood"]

    #model
    svr = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
    svr.fit(X_train, y_train)

    #predict
    y_pred = svr.predict(X_test)

    #evaluation 
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"Mean Absolute Error: {mae:.2f}")

    
    

/var/folders/gv/_x9lnydj1d71rl0vtl5mrtg00000gn/T/ipykernel_1550/1343164559.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['day_of_week'] = data['time'].dt.dayofweek
/var/folders/gv/_x9lnydj1d71rl0vtl5mrtg00000gn/T/ipykernel_1550/1343164559.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('time', axis=1, inplace=True)


Mean Squared Error: 0.95
Mean Absolute Error: 0.76
Mean Squared Error: 0.36
Mean Absolute Error: 0.46
Mean Squared Error: 0.41
Mean Absolute Error: 0.45
Mean Squared Error: 0.41
Mean Absolute Error: 0.51
Mean Squared Error: 0.41
Mean Absolute Error: 0.48
